In [1]:
import matplotlib
matplotlib.use('Agg')

import sys
sys.settrace
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pylab import *
import struct
import array
import os
import glob
import h5py
from scipy.interpolate import griddata

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
#plt.rcParams.update({
#    "text.usetex": True,
#    "font.family": "serif",
#    "font.serif": ["Palatino"],
#})

from importlib import reload

In [2]:
import athena_read

In [3]:
def FitBlackBody(nu_t):
  integral = 0.0;
  nu_2 = nu_t * nu_t;
  nu_3 = nu_t * nu_2;
  if nu_t < 1.5:
    integral = 0.051329911273422 * nu_3 -0.019248716727533 * nu_t * nu_3 + 0.002566495563671 * nu_2 * nu_3;
  else:
    exp_nu = exp(-nu_t);
    integral = -0.156915538762850 * exp_nu * (nu_3 + 2.69 * nu_2 + 6.714 * nu_t) + 1.000009331428801*(1- exp_nu);
  

  return integral;

In [17]:
def PlotProfile(datax, datay, xmin, xmax, ymin, ymax,  ylabel, label1, filename, xlabel='$r/r_g$', logscale=0, 
                xlogscale=0,datay1_2=None, datay1_3=None, datax2=None, datay2=None, datay2_2=None, datay2_3=None, 
                datax3=None, datay3=None, datay3_2=None, datay3_3=None, datax4=None, datay4=None, 
                datax5=None, datay5=None, label2='', label3='', label4='', label5='',title=None):
    plots, axes = plt.subplots(figsize=(9,11),dpi=300)
    plt.xlabel(xlabel, size = 35)
    plt.ylabel(ylabel, size = 35)
    plt.subplots_adjust(left=0.15,right=0.88,top=0.9,bottom=0.1)
    plt.ylim([ymin,ymax])
    plt.xlim([xmin,xmax])
    if logscale > 0:
      axes.set_yscale('log')
    if xlogscale > 0:
      axes.set_xscale('log')
    if title is not None:
      plt.title(title,size=25)

    plt.plot(datax,datay,color='magenta',marker='o',fillstyle='none',markersize=8,label=label1,linewidth=3.0)
    if datay1_2 is not None:
      plt.plot(datax,datay1_2,color='blue',linestyle='dotted',linewidth=4.0,alpha=1.0)
    if datay1_3 is not None:
      plt.plot(datax,datay1_3,color='black',linestyle='dashed',linewidth=4.0)
    if datay2 is not None:
      plt.plot(datax2,datay2,color='black',label=label2,linewidth=2.0,alpha=1.0,marker='x',fillstyle='none',markersize=8)
    if datay2_2 is not None:
      plt.plot(datax2,datay2_2,color='black',linestyle='dashed',linewidth=2.0,alpha=1.0)
    if datay2_3 is not None:
      plt.plot(datax2,datay2_3,color='red',linestyle='dashed',linewidth=4.0)
    if datay3 is not None:
      plt.plot(datax3,datay3,color='green',label=label3,marker='s',fillstyle='none',markersize=8,linewidth=3.0)
    if datay3_2 is not None:
      plt.plot(datax3,datay3_2,color='black',linestyle='dashed',linewidth=2.0,alpha=1.0)
    if datay3_3 is not None:
      plt.plot(datax3,datay3_3,color='green',linestyle='dashed',linewidth=4.0)
    if datay4 is not None:
      plt.plot(datax4,datay4,color='red',linestyle='dotted',label=label4,linewidth=3.0)
    if datay5 is not None:
      plt.plot(datax5,datay5,color=tableau20[0],label=label5,linewidth=2.0)      
    plt.legend(loc='lower right',bbox_to_anchor=(0.8, 0.3),frameon=False,fontsize = 35)
    axes.set_aspect('auto')
    axes.yaxis.set_tick_params(labelsize=25)
    axes.xaxis.set_tick_params(labelsize=25)
    plt.savefig(filename)
    plt.close(plots)


In [5]:
files=sorted(glob.glob('Data/thermal*athdf'))
num_file=len(files)

In [28]:
#for filename in files:
nu_grid=[0,0.75,2.5]
histories=np.zeros((num_file,5))

In [29]:
Prat=1
Crat=10
count=0
for filename in files:
  with h5py.File(filename, 'r') as f:
    attributes = f.attrs.items()
    attrs = dict(attributes)
    level = f.attrs['MaxLevel']
    time = f.attrs['Time']
  subsample = False

  data = athena_read.athdf(filename, level=level, subsample=subsample)
  histories[count,0]=time
  histories[count,1]=np.average(data['Er_0'])
  histories[count,2]=np.average(data['Er_1'])
  histories[count,3]=np.average(data['Er_2'])
  histories[count,4]=np.average(data['press']/data['rho'])
  count=count+1


In [30]:
ylabel='$E_r, a_rT^4$'
filename='thermal_relaxation.pdf'
xlabel='$t$'
label2='$E_{r,2}$'
label3='$E_{r,3}$'
label4='$a_rT^4$'
#print(histories[:,0])
PlotProfile(histories[:,0], histories[:,1],-0.2, 5.2, 1.e-2, 100, ylabel, '$E_{r,1}$', filename, xlabel, logscale=1,
           datax2=histories[:,0],datay2=histories[:,2],label2=label2,datax3=histories[:,0],datay3=histories[:,3],
            label3=label3,datax4=histories[:,0],datay4=histories[:,4]**4,label4=label4)

In [31]:
tg=histories[num_file-1,4]
emission=np.zeros(3)
emission[0]=FitBlackBody(nu_grid[1]/tg)
emission[1]=FitBlackBody(nu_grid[2]/tg)-FitBlackBody(nu_grid[1]/tg)
emission[2]=1-FitBlackBody(nu_grid[2]/tg)
Er1=histories[num_file-1,1]
Er2=histories[num_file-1,2]
Er3=histories[num_file-1,3]

In [32]:
print(emission*tg**4,Er1,Er2,Er3)
print(emission,tg)

[5.37646228e-02 1.49400713e+00 5.59637616e+01] 0.05376462638378143 1.067147970199585 56.24833679199219
[9.34849407e-04 2.59775221e-02 9.73087628e-01] 2.753840208053589
